In [1]:
import sys
sys.path.append(r'/home/songbird/Dropbox/Work/MDGAN_paper/code/Gagan/Train')
from utils import *
from hmmlearn import *
import numpy as np
import torch
import os
from time import time
from sklearn.externals import joblib
import pdb

In [2]:
import pdb

In [3]:
from glob import glob

In [4]:
basepath2models = '/media/songbird/datapartition/HMM/trained_models/merge_6000'
tutor_path = '/media/songbird/datapartition/HMM/trained_models/tutor/models/100h_gauss_diag.pkl'

pupil_paths = [glob(os.path.join(basepath2models, 'm0_b7r16','models', 'm0_150h_gauss_diag.pkl'))[0],
                glob(os.path.join(basepath2models, 'm1_b7r16','models', 'm1_150h_gauss_diag.pkl'))[0],
               glob(os.path.join(basepath2models, 'm2_b7r16','models', 'm2_130h_gauss_diag.pkl'))[0],
               glob(os.path.join(basepath2models, 'm3_b7r16','models', 'm3_150h_gauss_diag.pkl'))[0],
               glob(os.path.join(basepath2models, 'm4_b7r16','models', 'm4_150h_gauss_diag.pkl'))[0],
               glob(os.path.join(basepath2models, 'm5_b7r16','models', 'm5_150h_gauss_diag.pkl'))[0],
               glob(os.path.join(basepath2models, 'm6_b7r16','models', 'm6_150h_gauss_diag.pkl'))[0]]

zseq_folders = ['day9_b7r16', 'day12_b7r16', 'day16_b7r16', 'day19_b7r16', 'day22_b7r16', 'day25_b7r16', 'day28_b7r16']

bird_path = '/home/songbird/data/b7r16_val'

In [5]:


def viterbi_sample(X,model,covtype='diag'):
    #X is a sequence
    #calculate most likely path given X under the model
    #return a sample from that path
    _, Z = model.decode(X)#contains information for all states
    L=len(Z)-1 #all states minus the start-state which is not defined in our case
    sequence=[]
    #take a sample from each state-emission
    for i in range(L):
        mean_i = model.means_[Z[i+1]]
        if covtype=='diag':
            sigma_i = model.covars_[Z[i+1]]
        elif covtype=='tied':
            sigma_i = model.covars_
        elif covtype=='spherical':
            sigma_i = model.covars_[Z[i+1]]
            sigma_i = np.diag(np.repeat(sigma_i, repeats=X.shape[1]))
        else:
            # case "full"
            sigma_i = model.covars_[Z[i+1]]
        
        #sequence.append(np.random.multivariate_normal(mean_i,sigma_i,size=1))
        sequence.append(mean_i)
    # print (np.asarray(sequence).shape)
    return sequence


def rescale_spectrogram(s):
    if np.min(s) < 0:
        s = s - np.min(s) 
    s = s/np.max(s)
    return 10*np.log10(s + 0.01)


def decode(zhat, netG, method=1, give_audio = False, imageH=129, imageW=16, cuda = False):
    '''
    Input zhat should have correct number of steps, i.e. if the spectrogram to be decoded has 
    X time frames, then you should input the vector z[: round(X / imageW), :]. This is due to 
    batch_size effects in pytorch!
    '''
    if type(zhat)==np.ndarray:
        zhat = torch.from_numpy(zhat).float()
        zhat = zhat.resize_(zhat.shape[0], zhat.shape[1], 1, 1)
    if cuda:
        zhat = zhat.cuda()
    out_shape = [imageH, imageW]
    reconstructed_samples = []
    reconstruction = netG(zhat)
     
    for k in range(reconstruction.data.cpu().numpy().shape[0]):
        if method==1:
            reconstructed_samples.append(reconstruction.data[k,:,:,:].cpu().numpy().reshape(out_shape))
        elif method==2:
            reconstructed_samples.append( \
                                        inverse_transform(reconstruction.data[k,:,:,:].cpu().numpy().reshape(out_shape), N=500))
        elif method==3:
            reconstructed_samples.append(get_spectrogram(rescale_spectrogram(reconstruction.data[k,:,:,:].cpu().numpy().reshape(out_shape))))
        
    reconstructed_samples = np.concatenate(reconstructed_samples, axis=1)
    if give_audio and method==2:
        reconstructed_audio = lc.istft(reconstructed_samples)
    else:
        reconstructed_audio = []
    return rescale_spectrogram(reconstructed_samples), reconstructed_audio


def decode_by_batch(zhat, netG,  batch_size = 64, imageH=129, imageW=16, cuda = False):
    if type(zhat)==np.ndarray:
        zhat = torch.from_numpy(zhat).float()
        zhat = zhat.resize_(zhat.shape[0], zhat.shape[1], 1, 1)
    if cuda:
        zhat = zhat.cuda()
    out_shape = [imageH, imageW]
    reconstructed_samples = []
    # do inference in batches
    nbatches = round(zhat.size(0)/batch_size)
    i = 0
    for n in range(nbatches):
        reconstruction = netG(zhat[i:i+batch_size])
        i += batch_size
        for k in range(reconstruction.data.cpu().numpy().shape[0]):
            reconstructed_samples.append(reconstruction.data[k,:,:,:].cpu().numpy().reshape(out_shape))
    reconstructed_samples = np.concatenate(reconstructed_samples, axis=1)
    return rescale_spectrogram(reconstructed_samples)
    
def load_z_and_remove_redundant(path2fls):
    fls = os.listdir(path2fls)
    zseqs = []
    for f in fls:
        z = np.load(os.path.join(path2fls, f))
        zprev = z[0]
        seq = []
        seq.append(zprev)
        for i in range(1,len(z)):
            if np.sum(z[i]-zprev)!=0.0:
                seq.append(z[i])
                zprev = z[i]
            else:
                break
        seq = np.array(seq)
        zseqs.append(seq)
    return zseqs


def create_merged_sets(zseqs, required_len = 100):
    step_cntr = 0
    allseq = []
    seq = []
    for i in range(len(zseqs)):
        if step_cntr < required_len:
            seq.append(zseqs[i])
            step_cntr += zseqs[i].shape[0]
        else:
            allseq.append(np.concatenate(seq, axis=0))
            seq = []
            step_cntr = 0
    return allseq


def get_random_zsequence_sample_from_bird(birdpath, folder_names, Nsamps = 50):
    folder_zseq = []
    for f in folder_names:
        path2files = os.path.join(birdpath, f, 'z_sequences/')
        allfiles = os.listdir(path2files)
        idx = np.random.choice(len(allfiles),size=Nsamps,replace=False)
        zseq = [None for i in range(len(idx))]
        for k in range(len(idx)):
            # load z-sequence
            z = np.load(os.path.join(path2files, allfiles[k]))
            zprev = z[0]
            seq = []
            seq.append(zprev)
            for i in range(1,len(z)):
                if np.sum(z[i]-zprev)!=0.0:
                    seq.append(z[i])
                    zprev = z[i]
                else:
                    break
            seq = np.array(seq)
            zseq[k] = seq
        folder_zseq.append(zseq)
    return folder_zseq


def tempered_sampling(model, beta=3., timesteps=64, sample_obs=True, start_state_max=False):
    # start probability sample
    if start_state_max:
        row = np.argmax(model.startprob_)
    else:
        # choose a start state
        p = np.exp(beta * np.log(model.startprob_))
        p /= np.sum(p)
        s0 = np.random.multinomial(1,p)
        row = np.where(s0==1.)[0][0]
    s0 = row
    states = np.zeros((timesteps),dtype='int64')
    obs = np.zeros((timesteps, model.means_.shape[-1]))
    for i in range(timesteps):
        # extract the correct row from the transition matrix
        a = model.transmat_[row,:]
        # make the gibbs probability vector
        p = np.exp(beta * np.log(a))
        p /= np.sum(p)
        # sample from it 
        s = np.random.multinomial(1,p)
        row = np.where(s==1.)[0][0]
        states[i] = row
        # sample from the corresponding distribution in the model
        mean_i = model.means_[row]
        sigma_i = model.covars_[row]
        if sample_obs:
            # sample an observation 
            obs[i] = np.random.multivariate_normal(mean_i,sigma_i,size=1)
        else:
            obs[i] = mean_i
    return obs, states, s0


def sample_and_compute_single_score(model1, model2, beta,  timesteps, sample_obs, start_state_max):
    z,_,_ = tempered_sampling(model1, beta, timesteps, sample_obs, start_state_max)
    m1score = model1.score(z)
    m2score = model2.score(z)
    dpq = (1/timesteps)*(m1score - m2score)
    
    # now do dqp
    z,_,_ = tempered_sampling(model2, beta, timesteps, sample_obs, start_state_max)
    m2score = model2.score(z)
    m1score = model1.score(z)
    dqp = (1/timesteps)*(m2score - m1score)
    
    # symmetric score Jensen Shannon
    js = 0.5*dpq + 0.5*dqp
    return dpq, dqp, js


def compute_single_score_from_observation(zseq, model1, model2):
    '''
    Zseq is a sequence of latent variable vectors [timesteps x ndim]
    '''
    timesteps = zseq.shape[0]
    m1score = model1.score(zseq)
    m2score = model2.score(zseq)
    dpq = (1/timesteps)*(m1score - m2score)
    
    # now do dqp
    dqp = (1/timesteps)*(m2score - m1score)
    
    # symmetric score Jensen Shannon
    js = 0.5*dpq + 0.5*dqp
    return dpq, dqp, js


def juang_rabiner_HMM_dist(model1, model2, zseq_data = [], min_len = 40, beta = 1., Nsamp = 100000, \
                           timesteps = np.arange(start=40, stop=60, step=1), do_parallel=True):
    '''
    DKL(P||Q), DKL(Q||P), JS(P,Q) for two cases:
        1. When actual data is available (zseq_data)
        2. When samples are generated from model1 (P), assumed to be tutor
    '''
    Scores_per_len = []
    pdb.set_trace()
    if zseq_data:
        # for each given sequence, get scores (DKL[P||Q],DKL[Q||P],JS(P,Q))
        score = []
        for i,z in enumerate(zseq_data):
            score.append(compute_single_score_from_observation(z, model1, model2))
            if i%10==9:
                print('# %d/%d data sequences processed '%(i/len(zseq_data)))
        Scores_per_len.append([(np.mean([s[0] for s in score]),np.var([s[0] for s in score])), \
                               (np.mean([s[1] for s in score]),np.var([s[1] for s in score])), \
                               (np.mean([s[2] for s in score]),np.var([s[2] for s in score]))])
    else:
        for k in timesteps:
            if not do_parallel:
                score = []
                for n in range(Nsamp):
                    score.append(sample_and_compute_single_score(model1, model2, beta, k, sample_obs=False, start_state_max=True))
            else:
                score = Parallel(n_jobs=-3)(delayed(sample_and_compute_single_score)(model1, model2, \
                                                                                     beta, k, \
                                                                                     sample_obs=False, \
                                                                                     start_state_max=True) for n in range(Nsamp))
        
            # take the average score per length
            Scores_per_len.append([(np.mean([s[0] for s in score]),np.var([s[0] for s in score])), \
                                   (np.mean([s[1] for s in score]),np.var([s[1] for s in score])), \
                                   (np.mean([s[2] for s in score]),np.var([s[2] for s in score]))])
        if k%10==9:
            print('# %d/%d time steps simulated'%(k,len(timesteps)))
            
    return Scores_per_len


def compute_LL_HMM_dist(pupil_paths, tutor_path, birdpath = [], zseq_folders = [], pupdatasamps=50, beta = 3., Nsamp=100000, \
                        seq_lengths = np.arange(start=40, stop=60, step=1), do_parallel=False):
    tutorhmm = joblib.load(tutor_path)
    if zseq_folders:
        zseq_data = get_random_zsequence_sample_from_bird(birdpath, zseq_folders, Nsamps = pupdatasamps)
    scores = []
    pdb.set_trace()
    for i,p in enumerate(pupil_paths):
        start = time()
        print('\n ....... pupil merge number %d/%d .......'%(i,len(pupil_paths)))
        pupilhmm = joblib.load(p)
        per_day_score_list = juang_rabiner_HMM_dist(tutorhmm, pupilhmm, zseq_data[i], beta, Nsamp, seq_lengths, do_parallel)
        scores.append(per_day_score_list) 
        end = time()
        print('\n ....... time taken for this merge = %5d secs........ '%(end-start))
    return scores


In [6]:
scores = compute_LL_HMM_dist(pupil_paths,tutor_path,bird_path,zseq_folders,beta=1.)

> <ipython-input-5-08ac6d953720>(253)compute_LL_HMM_dist()
-> for i,p in enumerate(pupil_paths):
> <ipython-input-5-08ac6d953720>(254)compute_LL_HMM_dist()
-> start = time()
> <ipython-input-5-08ac6d953720>(255)compute_LL_HMM_dist()
-> print('\n ....... pupil merge number %d/%d .......'%(i,len(pupil_paths)))

 ....... pupil merge number 0/7 .......
> <ipython-input-5-08ac6d953720>(256)compute_LL_HMM_dist()
-> pupilhmm = joblib.load(p)
> <ipython-input-5-08ac6d953720>(257)compute_LL_HMM_dist()
-> per_day_score_list = juang_rabiner_HMM_dist(tutorhmm, pupilhmm, zseq_data[i], beta, Nsamp, seq_lengths, do_parallel)
--Call--
> <ipython-input-5-08ac6d953720>(214)juang_rabiner_HMM_dist()
-> def juang_rabiner_HMM_dist(model1, model2, zseq_data = [], min_len = 40, beta = 1., Nsamp = 100000,                            timesteps = np.arange(start=40, stop=60, step=1), do_parallel=True):
> <ipython-input-5-08ac6d953720>(220)juang_rabiner_HMM_dist()
-> Scores_per_len = []
> <ipython-input-5-08ac6d95

BdbQuit: 